In [2]:
!pip install snscrape

In [3]:
import pandas as pd
import itertools
import snscrape.modules.twitter as sntwitter


In [4]:
#Importing the datetime to calculate the time for scraping the 50000 tweets
from datetime import datetime

start_time = datetime.now()

#Creating dataframe called 'data' and storing the tweets from May 1st 2021 to 30th Juy 2021 for 'Vaccine'

data = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '"#vaccine since:2021-05-01 until:2021-07-30"').get_items(), 50000))
end_time = datetime.now()

#Printing the time duration for scraping these tweets
print('Duration: {}'.format(end_time - start_time))


Duration: 0:08:23.799330


In [10]:
data.columns

Index(['url', 'date', 'content', 'id', 'username', 'outlinks', 'outlinksss',
       'tcooutlinks', 'tcooutlinksss'],
      dtype='object')

In [7]:
data.to_csv("tweet.csv")

In [29]:
#keeping only date, id, content, user, lang, and hashtag and stored into dataframe called 'data'
df = data[['date', 'id', 'content', 'username']]
df1 = df.head(10000)

In [10]:
from transformers import pipeline
sentiment_classifier = pipeline('sentiment-analysis')

In [16]:
tweets = df1['content'].tolist()
tweets[2]

'@DAVEWILLISON1 Medical autonomy is a right Dave. These so called vaccines are still in trials until 2023. What’s happening is blatant coercion disguised as a solution to freedom in a free country. Less than 20% of Aussies have had it, and I can’t see us getting anywhere the numbers they want.'

In [17]:
df1['content'][2]

'@DAVEWILLISON1 Medical autonomy is a right Dave. These so called vaccines are still in trials until 2023. What’s happening is blatant coercion disguised as a solution to freedom in a free country. Less than 20% of Aussies have had it, and I can’t see us getting anywhere the numbers they want.'

In [19]:
results = sentiment_classifier(tweets)

In [30]:
#Calculating the sentiments
df1 = (
    df1
    .assign(sentiment = lambda x: x['content'].apply(lambda s: sentiment_classifier(s)))
    .assign(
         label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
         score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
)

In [60]:
df1.head(10)

,date,id,content,username,sentiment,label,score
0,2021-07-29 23:59:59+00:00,1420896922278240256,Act Now America: Say NO To Vaccine Passports h...,datrufwilfreeu,"[{'label': 'NEGATIVE', 'score': 0.993231892585...",NEGATIVE,0.993232
1,2021-07-29 23:59:59+00:00,1420896922181804032,So they are forcing my sister-in-law to take t...,ignacio1981sala,"[{'label': 'NEGATIVE', 'score': 0.998433232307...",NEGATIVE,0.998433
2,2021-07-29 23:59:59+00:00,1420896920453599234,@DAVEWILLISON1 Medical autonomy is a right Dav...,Bdj68Brad,"[{'label': 'NEGATIVE', 'score': 0.709976196289...",NEGATIVE,0.709976
3,2021-07-29 23:59:58+00:00,1420896917958057984,@_AlexHirsch The fact people are arguing under...,DrSallan_,"[{'label': 'POSITIVE', 'score': 0.894606113433...",POSITIVE,0.894606
4,2021-07-29 23:59:58+00:00,1420896917316272130,Find out which vaccine is seeing the most brea...,einerdrake,"[{'label': 'NEGATIVE', 'score': 0.991683542728...",NEGATIVE,0.991684
5,2021-07-29 23:59:58+00:00,1420896915831640069,@ncanarchist @AstorAaron @paolo_lim I am also ...,JakeHicks88,"[{'label': 'NEGATIVE', 'score': 0.997844457626...",NEGATIVE,0.997844
6,2021-07-29 23:59:58+00:00,1420896915709857798,"@ctpierson chilling, isn't it? like a doll's eyes",rebeccawatson,"[{'label': 'NEGATIVE', 'score': 0.920101523399...",NEGATIVE,0.920102
7,2021-07-29 23:59:57+00:00,1420896912845197312,@satyrntracks YAY THERAPY and vaccine,AxelSteele90x,"[{'label': 'NEGATIVE', 'score': 0.987305700778...",NEGATIVE,0.987306
8,2021-07-29 23:59:57+00:00,1420896912178302978,@MaxKolbe16 @Oilfield_Rando I mean this all st...,WithEternityIM,"[{'label': 'NEGATIVE', 'score': 0.999108076095...",NEGATIVE,0.999108
9,2021-07-29 23:59:57+00:00,1420896911679266823,@KatiePavlich @mikandynothem @MrsPerrin We kno...,madwoman1949,"[{'label': 'NEGATIVE', 'score': 0.999547123908...",NEGATIVE,0.999547


In [65]:
df1['content'][9]

'@KatiePavlich @mikandynothem @MrsPerrin We know only very recently did #McConnell and a minority of #TrumpGOP openly promote vaccines, faced w/indisputable science: 95%+ of current #Covid19 hospitalizations are unvaccinated. Predictable Trumper equivocations are increasingly less effective, thankfully. @MtnMD @jrehling'

In [57]:
#Visualizing the sentiments

import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Bar(x = df1["score"],
                    y = df1["label"],
                orientation = "h")) #set orientation to horizontal because we want to flip the x and y-axis

fig.update_layout(plot_bgcolor = "white")              
                                #apply our custom category order

fig.show()